## 149: Verificación de Cuenta vía Correo Electrónico con Token Temporal

Implementar el sistema de verificación de cuenta por correo electrónico, utilizando un token único y temporal enviado al registrarse.

🧩 1. Añadir campos al modelo User
python
Copiar
Editar
class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True, index=True)
    email = Column(String, unique=True, index=True)
    hashed_password = Column(String)
    is_active = Column(Boolean, default=False)
    verification_token = Column(String, nullable=True)  # 🆕 Token de verificación
    token_expiration = Column(DateTime, nullable=True)   # 🆕 Expiración del token
🧩 2. Registro de usuario con token de verificación
python
Copiar
Editar
@app.post("/registro")
def registrar_usuario(email: str, password: str, background_tasks: BackgroundTasks):
    if db.query(User).filter_by(email=email).first():
        raise HTTPException(status_code=400, detail="Ya existe un usuario con ese correo")

    hashed = pwd_context.hash(password)
    token = secrets.token_urlsafe(32)
    expiration = datetime.utcnow() + timedelta(hours=1)

    nuevo_usuario = User(
        email=email,
        hashed_password=hashed,
        is_active=False,
        verification_token=token,
        token_expiration=expiration
    )
    db.add(nuevo_usuario)
    db.commit()

    background_tasks.add_task(enviar_correo_verificacion, email, token)
    return {"mensaje": "📧 Verifica tu cuenta desde tu correo"}
🧩 3. Endpoint para verificar cuenta con el token
python
Copiar
Editar
@app.get("/verificar-cuenta")
def verificar_cuenta(token: str):
    user = db.query(User).filter_by(verification_token=token).first()

    if not user:
        raise HTTPException(status_code=404, detail="Token inválido")

    if user.token_expiration < datetime.utcnow():
        raise HTTPException(status_code=400, detail="Token expirado")

    user.is_active = True
    user.verification_token = None
    user.token_expiration = None
    db.commit()

    return {"mensaje": "✅ Cuenta verificada exitosamente"}
🧩 4. Función para enviar el correo de verificación
python
Copiar
Editar
def enviar_correo_verificacion(destinatario: str, token: str):
    link = f"https://tusitio.com/verificar-cuenta?token={token}"
    asunto = "✅ Verifica tu cuenta"
    cuerpo = f"Hola 👋, haz clic en este enlace para verificar tu cuenta:\n\n{link}\n\nEste enlace expira en 1 hora."
    # Lógica para enviar el correo aquí